In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

import geopandas as gpd
import shapely
import geojsonio
import geojson
import json
import pandas as pd
import numpy as np

from shapely.geometry import Point, Polygon, MultiPolygon
import shapely.wkt

import warnings
warnings.filterwarnings("ignore")

# pip install descartes

In [ ]:
airbnb = pd.read_csv("airbnb.csv")

In [ ]:
areas_list = requests.get("http://mapakriminality.cz/api/areas/?level=3") # level 3 = local departments

In [ ]:
crime_list = requests.get("http://mapakriminality.cz/api/crimetypes")

In [ ]:
crime_retrieve = requests.get("http://mapakriminality.cz/api/crimes?crimetypes=103-903&timefrom=6-2019&timeto=6-2019&groupby=time")

In [ ]:
print(areas_list.status_code)
print(crime_list.status_code)
print(crime_retrieve.status_code) # everything went okay

# Areas

In [ ]:
# First we will deal with Areas, which define the police destricts (geographically) in Prague

In [ ]:
areas_list = areas_list.json() 

In [ ]:
gdf_areas = gpd.GeoDataFrame(areas_list) # To select the right values we need, firstly, we transform the json (dict) 
                                         # to geopandas data frame
    
sub_dict_areas = dict(gdf_areas["areas"]) # We are interested only in "area" column, which seems to contain information we need
sub_dict_areas_df = gpd.GeoDataFrame(sub_dict_areas) 

sub_dict_areas_df = sub_dict_areas_df.T # Transpose so we get the right df we need

In [ ]:
sub_dict_areas_df["Code"] = sub_dict_areas_df ["Code"].astype(int)
# The codes which starts with 00 are Prague Codes
# When change them to integers, Prague codes are all that are smaller than 10000
# Example: Prague 1 - 001110 --> 1110
# First non-prague starts with 01
# Therfore first non-Prague code is 010110 --> 10110

In [ ]:
# Get only Prague criminality districts
Prague_areas_crime = sub_dict_areas_df[sub_dict_areas_df.Code < 10000]

# Dataset for further cleaning is ready

In [ ]:
Prague_areas_crime.head()

In [ ]:
# These three rows has different syntax, as it refers to two different polygons each, I need to clean these rows manually
Prague_areas_crime["Geometry"][17]
Prague_areas_crime["Geometry"][21] 
Prague_areas_crime["Geometry"][46]

In [ ]:
# To get the right format of coordinates, first make the coordinates pairs
Prague_areas_crime["Geometry"] = Prague_areas_crime["Geometry"].replace({' ':':'}, regex = True).replace({',':' '}, regex = True).replace({':':','}, regex = True)

In [ ]:
# row 17 and 21 and 46 seems to be problematic as they have different syntax
smichov_row = Prague_areas_crime.loc[17]
zlicin_row = Prague_areas_crime.loc[21]
hostivar_row = Prague_areas_crime.loc[46]

In [ ]:
# Drop it from initial df and work with them separately, at the end will be concat together with original df
Prague_areas_crime = Prague_areas_crime.drop(17)
Prague_areas_crime = Prague_areas_crime.drop(21)
Prague_areas_crime = Prague_areas_crime.drop(46)

In [ ]:
# Row 17 - values above were manually withdrawn from the dataset
smichov1 = "POLYGON((14.409178398997042 50.081354184055861,14.410422357270217 50.072751734624369,14.415876915137783 50.061879901103779,14.412800011056063 50.050755121777108,14.409154281856626 50.051732535204494,14.407198433879673 50.055050456842068,14.407781755436524 50.048860650324407,14.373282171272052 50.055481599298602,14.376961664737101 50.055694634120158,14.378222146571394 50.05580440227957,14.379773224495427 50.05614594867113,14.396001213617094 50.066144790677022,14.400293223496835 50.069938384635627,14.399823382707339 50.071802696207477,14.399148561308387 50.072977271253109,14.39803151301725 50.074439163923941,14.397096792204556 50.075182148370587,14.392565416549395 50.078159828506834,14.397650559141766 50.080001756346491,14.409178398997042 50.081354184055861))"
smichov2 = "POLYGON((14.373282171272052 50.055481599298602,14.373282015714192 50.05548159028492,14.373282154053754 50.055481602596409,14.373282171272052 50.055481599298602))"

# Tranforming values into the shapely format
smichov_pol1 = shapely.wkt.loads(smichov1)
smichov_pol2 = shapely.wkt.loads(smichov2)

# Visual analysis of these values
p1 = gpd.GeoSeries(smichov_pol1) #
p1.plot(color = "lightblue")
plt.savefig("p1.png")

p2 = gpd.GeoSeries(smichov_pol2) # Really small area
p2.plot(color='red')
plt.show()

In [ ]:
# Row 17 - continue

polygons_s = [smichov_pol1, smichov_pol2]

multi_pol_smichov = MultiPolygon(polygons_s) # Apply MultiPolygon function from shapely to get the right geometry

smichov_row["Geometry"] = multi_pol_smichov # Replace the orignal value with MultiPolygon value

smichov_row = (pd.DataFrame(smichov_row)).T # transpose to get the shape as the original dataframe

multi_pol_smichov

In [ ]:
# Row 21 - values above were manually copied from the datasets

zlicin1 = "POLYGON((14.29341075019954 50.077469677336246,14.292074225328276 50.075965629841505,14.291528717873765 50.075220429494827,14.291451560084571 50.075079700305068,14.291417081491748 50.075016784232027,14.291410301108694 50.07496080023941,14.291396114299864 50.074842603047955,14.291366537499698 50.074595479443005,14.291376776183196 50.074297367402188,14.291676742055603 50.071408859462416,14.291816894371054 50.070244825244544,14.292182099485872 50.069137014004959,14.292473709366842 50.068284413007298,14.293101195112325 50.066719895679611,14.293231447891449 50.066652415996622,14.304184106945414 50.06105191532901,14.304441220222238 50.060949948144419,14.30450927610865 50.060930627202858,14.305601770748835 50.061116913693077,14.306694771815522 50.06130741029132,14.309887216921595 50.061865909140522,14.31236200580175 50.062445676812494,14.312886426235583 50.062569144403966,14.313147724539208 50.062631839529828,14.313671276042619 50.062758039513831,14.313931116029348 50.06282096527427,14.314193358099534 50.062885830131655,14.322117357675419 50.065324824570318,14.322696866025172 50.06688895885268,14.325460177267889 50.074548570740198,14.341839075016683 50.07541618931625,14.339074086755117 50.056002482557865,14.274971669433048 50.049757957218532,14.27270685166563 50.050685839699966,14.272673025071745 50.050801544478681,14.272637922570505 50.050839241999292,14.272577695279598 50.050903731979432,14.257300590767215 50.054502763885544,14.250791408256188 50.056963487003998,14.248250709363349 50.058337033515592,14.24815613753592 50.058389341784121,14.247810052626999 50.059799876505657,14.247744827390614 50.061877583915923,14.258123839224279 50.071596541791649,14.289407212304765 50.077006366948012,14.293277648827168 50.077593187802769,14.29341075019954 50.077469677336246))"
zlicin2 = "POLYGON((14.289563345301277 50.057652074277669,14.289563936308477 50.057651904021071,14.28957971774812 50.057655318058281,14.289563345301277 50.057652074277669))"

zlicin_pol1 = shapely.wkt.loads(zlicin1)
zlicin_pol2 = shapely.wkt.loads(zlicin2)

polygons_z = [zlicin_pol1, zlicin_pol2]

multi_pol_zlicin = MultiPolygon(polygons_z)

zlicin_row["Geometry"] = multi_pol_zlicin

zlicin_row = (pd.DataFrame(zlicin_row)).T

multi_pol_zlicin

In [ ]:
# Row 46 - values above were manually withdrawn from the datasets
hostivar1 = "POLYGON((14.523099863328587 50.086725993887761,14.539091432255972 50.081029360504473,14.538421192684877 50.076980937457208,14.53767119709064 50.071717781591218,14.537595997146562 50.071139544382518,14.537597958672272 50.070196121638119,14.537721048982696 50.068945623870768,14.539987342789797 50.069162024911179,14.542686167655061 50.06951258468272,14.547333804725923 50.067036358923772,14.548475973710227 50.066389557537725,14.549175907623919 50.065910417859918,14.552551880949453 50.063583528567356,14.565537863340856 50.053275337726625,14.5671270047224 50.051953257636782,14.57104528329136 50.047527386190772,14.57199973653139 50.043983874342523,14.571524002993572 50.041076704798478,14.564941489611966 50.031384984265223,14.558829279940211 50.029571653738287,14.549348770420877 50.027594296897838,14.549462659130249 50.028027463829375,14.549616442096013 50.029509965013332,14.543640187670364 50.038110036445168,14.538832860508819 50.040086189211252,14.537915917460445 50.040173028314356,14.537765358520186 50.040179441889933,14.537330781454621 50.040161070392742,14.537115316266457 50.040145918800611,14.536828641546025 50.040125709167711,14.534141051833995 50.039795174338636,14.533935653970493 50.039751045145167,14.533582424998816 50.039669735107076,14.53319091788274 50.039533529344517,14.532895356839221 50.039430683089797,14.532931470651299 50.039320422146808,14.532950770127904 50.039247071004588,14.532918963802796 50.039130858484427,14.532772782272406 50.038919146081703,14.532709645459272 50.0388278638207,14.532260845135847 50.038610774467109,14.531876751621789 50.038462872125955,14.530740157067271 50.038250377369259,14.525678610525732 50.037345847138866,14.523890506963044 50.037393651423884,14.523667739811174 50.037400427740714,14.514527921044582 50.039607720269629,14.514445110348893 50.039632845969528,14.511382765298739 50.0406116073463,14.511268268535856 50.040673240768868,14.512870895671437 50.043411497835329,14.502712706621047 50.044529488983152,14.51779095481022 50.06148933216685,14.523318019096822 50.067086662292184,14.523577374600167 50.068827947108097,14.523582409775248 50.0700141959598,14.52357557799311 50.070243717087955,14.523507499416752 50.07088863731046,14.523463376323784 50.071299729878831,14.523406367413591 50.071689094750901,14.523165069996606 50.073099298552506,14.52311842475264 50.073344385787031,14.523075258781695 50.073496405171831,14.521811579392214 50.07639154069625,14.521301677152474 50.077490913574366,14.521195880866427 50.077568388645886,14.519589730814429 50.077646091688713,14.517262775186303 50.077752941898645,14.514397235405916 50.077882013708134,14.514214168089904 50.077890183775217,14.512336440687301 50.077935708344761,14.511976106178533 50.077941298560347,14.508780138761905 50.077851407081894,14.507273331591842 50.084161960065927,14.518463911575104 50.086576475667059,14.523084738708532 50.088014098671515,14.523099863328587 50.086725993887761))"
hostivar2 = "POLYGON((14.559717677974149 50.040154766994903,14.560669099051594 50.040055941510708,14.559842748789348 50.040141792477684,14.559717677974149 50.040154766994903))"

hostivar_pol1 = shapely.wkt.loads(hostivar1)
hostivar_pol2 = shapely.wkt.loads(hostivar2)

polygons_h = [hostivar_pol1, hostivar_pol2]

multi_pol_hostivar = MultiPolygon(polygons_h)

hostivar_row["Geometry"] = multi_pol_hostivar

hostivar_row = (pd.DataFrame(hostivar_row)).T

multi_pol_hostivar

In [ ]:
# The three above rows will be replaced manually in the dataframe

In [ ]:
# Clean the rest of the variables in the original dataframe
Prague_areas_crime["Geometry"]  = Prague_areas_crime["Geometry"].replace({'<Polygon><altitudeMode>relativeToGround</altitudeMode><outerBoundaryIs><LinearRing><altitudeMode>relativeToGround</altitudeMode><coordinates>':'POLYGON(('}, regex = True)
Prague_areas_crime["Geometry"]  = Prague_areas_crime["Geometry"].replace({'</coordinates></LinearRing></outerBoundaryIs></Polygon>':'))'}, regex = True)

In [ ]:
# apply "well-known-text" function to get the proper shapely Polygon structure
Prague_areas_crime["Geometry"] = Prague_areas_crime["Geometry"].apply(shapely.wkt.loads)

In [ ]:
# combine the dataset and the changed rows (multipolygons)
Prague_areas_crime_concat = pd.concat([Prague_areas_crime, smichov_row, zlicin_row, hostivar_row], axis = 0)

In [ ]:
# prepared dataset for further merging with the value of total criminality in June 2019
Prague_areas_crime_concat.to_csv("criminality_cleaned.csv")

In [ ]:
Prague_areas_crime_cleaned = pd.read_csv("criminality_cleaned.csv")

# Crime Types

In [ ]:
crime_list = crime_list.json()

In [ ]:
# We need the Total criminality for the study, which refers to code "101-903" (Total criminality)
crime_list["crimes"][0]

# Criminality index

In [ ]:
crime_retrieve = requests.get("http://mapakriminality.cz/api/crimes?crimetypes=103-903&timefrom=6-2019&timeto=6-2019&groupby=time")

In [ ]:
crime_retrieve = crime_retrieve.json() # To work with the retrieved data, we must treat them as json (the orignal format)
                                       # which is treated as dictionary in python

In [ ]:
# Example how to get the crime index from the api for specific district
# areacode = 001110 
# crimetypes = 101-903 (Total criminality)
# timefrom = 6-2019
# timeto = 6-2019
# groupby = area (not necessary in our case)
area1 = requests.get("http://mapakriminality.cz/api/crimes?areacode=001110&crimetypes=101-903&timefrom=6-2019&timeto=6-2019&groupby=area")
area1 = area1.json()
area1["crimes"][0]["CrimeRate"] # Criminality index

In [ ]:
# Iterate over all area_codes to recieve the criminality index
crime_indices = {}
for code in Prague_areas_crime_cleaned["Code"]: # Dont forget the "00" we deleted in the pre-processing
    url = "http://mapakriminality.cz/api/crimes?areacode=00" + str(code) + "&crimetypes=101-903&timefrom=6-2019&timeto=6-2019&groupby=area"
    crime_retrieve = requests.get(url)
    crime_retrieve = crime_retrieve.json()
    crime_index = crime_retrieve["crimes"][0]["CrimeRate"] # Criminality index
    if type(crime_index) == float:
        crime_indices[code] = crime_index
    elif type(crime_index) == str:
        crime_indices[code] = float(crime_index)
    else:
        crime_indices[code] = crime_index

In [ ]:
Prague_areas_crime_cleaned["criminality_index"] = Prague_areas_crime_cleaned["Code"] # Initiate column where we will map crime indices

In [ ]:
# As the dictionary is in the form code:criminality_index, we can use map function to map values into the column
# based on the index
Prague_areas_crime_cleaned.criminality_index = Prague_areas_crime_cleaned.criminality_index.map(crime_indices)

In [ ]:
print(len(crime_indices))
print(len(Prague_areas_crime_cleaned)) # one row is twice in our dataset
# it is row with code 1313

In [ ]:
print(len(Prague_areas_crime_cleaned["Code"].unique())) 
Prague_areas_crime_cleaned = Prague_areas_crime_cleaned.drop_duplicates(subset = ['Code'])

In [ ]:
# Crime maps defines 4 train station in Prague separately to measure the act individually on such places
# Places with end of code "50" or "51" 
# Train stations Masaryk station,Holešovické station, Wilson´s station, Smíchovské station 
# The Crime index is not counted, as it is recounted to people lives in the specific district,
# Hence cannot be recounted on the places like stations
# However, we can assigned a value of criminality manually, according to the city district/quarter where it is located
# Hence we will assume it belongs to the districts below:

# Masaryk Sation (1150) - Prague 1, Benediktinska Police district
# Holesovice Station (1151) - Prague 7, Holesovice Police District
# Wilson (Main) Station (1250) - Prague 1, Benediktinska Police district
# Smichov Station (1251)- Prague 5, Smíchov Police District

In [ ]:
Prague_areas_crime_cleaned.head()

In [ ]:
index_for_1111 = Prague_areas_crime_cleaned.loc[(Prague_areas_crime_cleaned["Code"] == 1111), "criminality_index"][1] # Benediktinska
index_for_1120 = Prague_areas_crime_cleaned.loc[(Prague_areas_crime_cleaned["Code"] == 1120), "criminality_index"][9] # Holesovice
index_for_1213 = Prague_areas_crime_cleaned.loc[(Prague_areas_crime_cleaned["Code"] == 1213), "criminality_index"][48] # Smichov

In [ ]:
Prague_areas_crime_cleaned.loc[Prague_areas_crime_cleaned.Code == 1150, 'criminality_index'] = index_for_1111 # Masaryk
Prague_areas_crime_cleaned.loc[Prague_areas_crime_cleaned.Code == 1151, 'criminality_index'] = index_for_1120 # Holesovice
Prague_areas_crime_cleaned.loc[Prague_areas_crime_cleaned.Code == 1250, 'criminality_index'] = index_for_1111 # Main/Wilson
Prague_areas_crime_cleaned.loc[Prague_areas_crime_cleaned.Code == 1251, 'criminality_index'] = index_for_1213 # Smichov

# Mapping crime index to each Airbnb listing

In [ ]:
airbnb["coordinates"] = list(zip(airbnb.longitude, airbnb.latitude)) 

In [ ]:
airbnb.coordinates = airbnb.coordinates.apply(Point) # make all of the tuples POINT, for purpose of using 
# shapely package

In [ ]:
# Shapely objects (Point, Polygons) cannot be saved in csv as shapely object
# They are saved as string, therefore for further analysis has to be converted
# airbnb["coordinates"] = airbnb["coordinates"].apply(shapely.wkt.loads) # Convert to Shapely Point object

In [ ]:
Prague_areas_crime_cleaned["Geometry"] = Prague_areas_crime_cleaned.Geometry.apply(shapely.wkt.loads)

In [ ]:
type(Prague_areas_crime_cleaned.Geometry[0])

In [ ]:
# To check if these rows are in Prague (therefore they are located on the edge of Polygons, which has not been yet analysed)
# We will use the geojson dat of the boundaries of Prague

boundaries = gpd.read_file("praha_hranice_uzemi.json") 
# Source: https://opendata.praha.eu/dataset/ipr-praha_-_hranice_uzemi

prague = gpd.GeoSeries(boundaries["geometry"][0]) # Really small area
prague.plot(color='lightblue')
plt.show()

In [ ]:
# Assign crime index to each airbnb listings, according to the polygon, where it is located
crime_indices_criminality = {}

for room_id, point in zip(airbnb["id"], airbnb["coordinates"]):

    for index, polygon in zip(Prague_areas_crime_cleaned["criminality_index"], Prague_areas_crime_cleaned["Geometry"]):

        if polygon.contains(point) == True:
            crime_indices_criminality[room_id] = index
            
        elif point.intersects(polygon) == True: # Check also the edges
            crime_indices_criminality[room_id] = index
            
        else:
            continue

In [ ]:
print(len(crime_indices_criminality))
len(airbnb) # some of the apartment are not within the Prague Police Districts
# We will map them and see what are these listings

In [ ]:
# Initiate the column where we will map values:
airbnb["criminality_index"] = airbnb["id"]

In [ ]:
# As the dictionary is in the form room_id : criminality_index, we can use map function to map values into the column
# based on the index
airbnb.criminality_index = airbnb.criminality_index.map(crime_indices_criminality)

In [ ]:
# Recieve indices of row, where nan
rows_with_nan_indices = airbnb.index[airbnb["criminality_index"].isnull()].tolist()

In [ ]:
# Rows with nan on criminality_index
is_nan = airbnb.isnull()
row_has_nan = is_nan.any(axis=1)
airbnb[row_has_nan]
# 9 listings are apparentely outside the definition of Prague Police Districts
# However, As we showed above, according to the data of Prague boundaries, all listings are in Prague,
# Therefore, they will not be deleted, and the nearest polygon value will be assigned.

In [ ]:
# Create MultiPolygon from all police district polygons
polygons_crime_list = []
for polygon in Prague_areas_crime_cleaned["Geometry"]:
    polygons_crime_list.append(polygon)
    
polygons = MultiPolygon(polygons_crime_list)

In [ ]:
# Find nearest polygon for each row
nan_polygon_list = []

for point in rows_with_nan_indices:
    min_poly = min(polygons, key = airbnb["coordinates"][point].distance)
    nan_polygon_list.append(min_poly)
    
# Inspect polygons format, then assign criminality_index for each of these polygons
dist = []
crime_index_list = []
for polygon in nan_polygon_list:
    for code, prague_poly, crime_index in zip(Prague_areas_crime_cleaned["Code"], Prague_areas_crime_cleaned["Geometry"], Prague_areas_crime_cleaned["criminality_index"]):
        if polygon == prague_poly:
            dist.append(code)
            crime_index_list.append(crime_index)
            
print(dist, rows_with_nan_indices, crime_index_list)

In [ ]:
# Assign the crime index according to the nearest polygon
for index, crime_index in zip(rows_with_nan_indices, crime_index_list):
    airbnb.iloc[index, airbnb.columns.get_loc("criminality_index")] = crime_index

In [ ]:
airbnb.drop(['Unnamed: 0'], axis=1)

In [ ]:
airbnb.to_csv("airbnb2.csv") # save the new dataset with criminality indices